In [49]:
import tensorflow as tf
import pandas as pd
from transformers import GPT2Config, TFGPT2LMHeadModel, GPT2Tokenizer
from tokenizers import CharBPETokenizer
from pathlib import Path
import torch
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoModel, BertTokenizerFast, GPT2Tokenizer, GPT2LMHeadModel, AdamW
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable
import random
import time, math, copy
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torchtext
from tokenizers import CharBPETokenizer
import tokenizers
import operator
from queue import PriorityQueue

import os

# specify GPU
device = torch.device("cuda")

# Import from external file
X_train = pd.read_csv("output/x_train.csv")
X_test = pd.read_csv("output/x_test.csv")

In [50]:
SOS_TOKEN = '<sos>'
EOS_TOKEN = '<eos>'

MAX_LEN = 5000

def len_filter(example):
    return len(example.src) <= MAX_LEN and len(example.tgt) <= MAX_LEN

In [51]:
# Set paths
train_path = 'output/x_train_nn.csv'
test_path = 'output/x_test_nn.csv'

# Save files
X_train[['question', 'prep_answer', 'cluster']].to_csv(train_path, index=False, header=False)
X_test[['question', 'prep_answer', 'cluster']].to_csv(test_path, index=False, header=False)

In [52]:
# the folder 'text' contains all the files
paths = [str(x) for x in Path("./data/").glob("**/*.csv")]
tokenizer = CharBPETokenizer(bert_normalizer=False)
# train the tokenizer model
tokenizer.train('output/x_train_nn.csv')
# saving the tokenized data in our specified folder
save_path = 'tokenized_data'
tokenizer.save_model(save_path)

['tokenized_data\\vocab.json', 'tokenized_data\\merges.txt']

In [53]:
# loading tokenizer from the saved model path
tokenizer = GPT2Tokenizer.from_pretrained(save_path, padding=True, truncation=True, return_tensors="tf")
tokenizer.add_special_tokens({
  "eos_token": "</s>",
  "bos_token": "<s>",
  "unk_token": "<unk>",
  "pad_token": "<pad>",
  "mask_token": "<mask>"
})
# creating the configurations from which the model can be made
config = GPT2Config(
  vocab_size=tokenizer.vocab_size,
  bos_token_id=tokenizer.bos_token_id,
  eos_token_id=tokenizer.eos_token_id
)
# creating the model
model = TFGPT2LMHeadModel(config)

In [54]:
single_string = ''
for filename in paths:
  with open(filename, "r", encoding='utf-8') as f:
   x = f.read()
  single_string += x + tokenizer.eos_token
string_tokenized = tokenizer.encode(single_string)

In [59]:
def encode_qa(questions, answers, tokenizer, maxlen=1024):
    """
    https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras
    """

    #tokenizer.truncate_sequences(maxlen=maxlen)
    #tokenizer.enable_padding(pad_id=1, pad_token = "<pad>")

    q_ids = []
    a_ids = []

    for i in range(0, len(questions)):
        q = questions[i]
        a = answers[i]

        q_id = tokenizer.encode(q)[:1]
        a_id = tokenizer.encode(a)[:36]

        if len(q_id) >= 1 and len(a_id) >= 36:
            q_ids.append(q_id)
            a_ids.append(a_id)
        #if len(encs.ids) > 512:
         #   return q, a

    return q_ids, a_ids

In [60]:
test_q = tokenizer.encode(list(X_train['question']))
test_a = tokenizer.encode(list(X_train['prep_answer']))
q_array, a_array = encode_qa(X_train['question'], X_train['prep_answer'], tokenizer)

In [41]:
examples = []
block_size = 100
BATCH_SIZE = 12
BUFFER_SIZE = 1000
for i in range(0, len(string_tokenized) - block_size + 1, block_size):
  examples.append(string_tokenized[i:i + block_size])
#print(examples)
inputs, labels = [], []
for ex in examples:
  inputs.append(ex[:-1])
  labels.append(ex[1:])
  #print(labels)
q_test = list(tf.ragged.constant(q_array))
dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

NameError: name 'string_tokenized' is not defined

In [58]:
q_array_tensor = tf.constant(X_train['cluster'])

In [61]:
#dataset_test = tf.data.Dataset.from_tensor_slices((tf.ragged.constant(q_array), tf.ragged.constant(a_array)))
dataset_test = tf.data.Dataset.from_tensor_slices((a_array, q_array))
#dataset_test = dataset_test.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [14]:
dataset_ans = tf.data.Dataset.from_tensor_slices((X_train['prep_answer']))
dataset_q = tf.data.Dataset.from_tensor_slices((X_train['question']))

In [63]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy()
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [64]:
num_epoch = 1
history = model.fit(dataset_test, epochs=num_epoch)

ValueError: in user code:

    d:\programdata\venv\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    d:\programdata\venv\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    d:\programdata\venv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    d:\programdata\venv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    d:\programdata\venv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    d:\programdata\venv\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
        outputs = model.train_s

In [ ]:
text = "trump has"
# encoding the input text
input_ids = tokenizer.encode(text, return_tensors='tf')
# getting out output
beam_output = model.generate(
  input_ids,
  max_length = 50,
  num_beams = 5,
  temperature = 0.7,
  no_repeat_ngram_size=2,
  num_return_sequences=5
)

In [ ]:
print(tokenizer.decode(beam_output[0]))